In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)

library("rstan")
options(mc.cores = 2)
rstan_options(auto_write = TRUE) # avoid recompilation of unchanged Stan programs

library("bayesplot")
library("dplyr")
library("ggplot2")

The data consists of average monthly temperatures from the FMI Kilpisjärvi station, for the last four decades. FMI has their own API, but casual downloads are easiest through their interactive service at https://ilmatieteenlaitos.fi/havaintojen-lataus#!/

Why Kilpisjärvi? Its a beautiful place, one of the few mountaineous areas in Finland. Its high latitude its temperature interesting, for warming due to greenhouse gases tends to be much more pronounced in the artic than on southern latitudes. The scripts on this notebook are general though, so that you can insert data from your favorite locations easily.

In [ ]:
d <- readr::read_csv("https://raw.githubusercontent.com/euxoa/ompeluseura/master/kilpisjarvi_raw.csv") %>% 
  setNames(c("year", "month", "day", "_clock", "tzone", "temp")) %>%
  mutate(t = ISOdate(year, month, day), 
         f_month = as.factor(month),
         decade = as.numeric(t - ISOdate(2000, 1, 1), units="days")/365.25) %>%
  select(year, f_month, t, decade, temp)

In [ ]:
ggplot(d, aes(x=t, y=temp)) +
    geom_line() +
    labs(title = "Monthly average temperature (C°)")

In [ ]:
ggplot(d, aes(x=t, y=temp, color=f_month)) +
    geom_line() +
    labs(title = "Monthly average by month", color= "Month")

An upward trend is obvious. Note high variance of the winter months!

In [ ]:
ggplot(d, aes(x=t, y=temp, color=f_month)) +
    geom_smooth(method="lm") +
    labs(title = "Trend by month", color = "Month")

In [ ]:
d_year <- d %>%
    group_by(year) %>%
    summarise(n=n(), temp=mean(temp), decade=mean(decade)) %>%
    filter(n==12)

In [ ]:
ggplot(d_year, aes(x=year, y=temp)) +
    geom_line() +
    labs(title = "Yearly average temperature (C°)")

In [ ]:
# Our calculated decade variable is just distance from 1.1.2000 in years. It's the time variable with a sane variance.
ggplot(d_year, aes(x=year, y=decade)) +
    geom_line() +
    labs(title = "Our calculated decade variable")

In [ ]:
stan_data_yearly <- with(d_year, list(N=length(temp), decade=decade, temp=temp))

In [ ]:
compile_and_fit_model <- function(model_code, data, vars_of_interest) {
  stan_start_time <- Sys.time()
  model <- stan_model(model_code = model_code) # compilation takes time if model is changed
  fit <- sampling(model, data = data)
  message("Compilation and fitting in secs ", difftime(Sys.time(), stan_start_time, units="secs"))
  
  posterior <- as.matrix(fit)
  print(traceplot(fit))
  print(fit)

  # Print posterior distributions for interesting variables
  posterior <- as.matrix(fit)
  for (var_name in vars_of_interest) {
    plot <- mcmc_areas(posterior, pars = c(var_name), prob = 0.8)  + 
      ggtitle(paste("Variable ", var_name, " posterior distributions with median and 80% interval"))
    print(plot)  
  }

  message("Total duration in secs ", difftime(Sys.time(), stan_start_time, units="secs"))
  return(fit)
}

The first model is a linear regression on the data aggregated to yearly level.

In [ ]:
model_simple_normal <- "
data {
  int<lower=0> N;
  vector[N] decade;
  vector[N] temp;
}
parameters {
  real a;
  real b;
  real<lower=0> sigma;
}
model {
  temp ~ normal(a + b * decade, sigma);
}
"
fit_sn <- compile_and_fit_model(model_code = model_simple_normal, data = stan_data_yearly, vars_of_interest = c("b"))


In the second model, we take monthly data, and fit linear regression lines to each of the months separately. 

In [ ]:
stan_data_monthly <- with(d, list(N=length(temp), decade=decade, month=as.integer(f_month), temp=temp))

In [ ]:
monthly_model_code <- "
data {
  int N;
  real decade[N];
  real temp[N];
  int month[N];
}
parameters {
  real<lower=0> sigma[12];
  real b[12];
  real k[12];
}
model {
  for (i in 1:N) {
     int m = month[i];
     temp[i] ~ normal(k[m] * decade[i] + b[m], sigma[m]); }
  sigma ~ normal(0, 5);
  b ~ normal(0, 5);
  k ~ normal(0, 1);
}
"
fit <- compile_and_fit_model(model_code = monthly_model_code, data = stan_data_monthly, vars_of_interest = c())

You can plot samples of the model, for example by variable pairs.

In [ ]:
data.frame(s1=extract(fit, "sigma[1]")[[1]], s2=extract(fit, "b[1]")[[1]]) %>%
    ggplot(aes(x=s1, y=s2)) +
    geom_point()

Below, the change of temperature per decade, by month. Note the higher uncertainty of winter months!

In [ ]:
plot(fit, pars="k")

Winter months also have higher residual variance.

In [ ]:
plot(fit, pars="sigma")

You can do hypothesis testing with the samples.

In [ ]:
dec_jun <- extract(fit, "k[12]")[[1]] - extract(fit, "k[6]")[[1]]
mean(dec_jun>0)
hist(dec_jun, n=100)

In [ ]:
monthly_model_code2 <- "
data {
  int N;
  real decade[N];
  real temp[N];
  int month[N];
}
parameters {
  real<lower=0> sigma[12];
  real b[12];
  real k[12];
  real<lower=0> sigma_change_k;
}
model {
  for (i in 1:N) {
     int m = month[i];
     temp[i] ~ normal(k[m] * decade[i] + b[m], sigma[m]); }
  sigma ~ normal(0, 5);
  b ~ normal(0, 5);
  for (i in 2:12)
     k[i] ~ normal(k[i-1], sigma_change_k);
  k[1] ~ normal(k[12], sigma_change_k);
  k[5] ~ normal(0, 1); // Any k will do to fix the overall level.
}
"
fit2 <- compile_and_fit_model(model_code = monthly_model_code2,
                              data = stan_data_monthly,
                              vars_of_interest = c())

In [ ]:
plot(fit2, pars="k")
plot(fit, pars="k")

In [ ]:
data.frame(s1=extract(fit2, "k[1]")[[1]], s2=extract(fit2, "k[2]")[[1]]) %>%
    ggplot(aes(x=s1, y=s2)) +
    geom_point()